# Лабораторная работа № 1

Преподаватель: Кондаратцев В.Л.

Студент: Горохов М.А.

Группа: М8О-109М-23


## Задание:
Градиентный спуск и его модификации
   - Выбрать тестовые функции оптимизации (https://ru.wikipedia.org/wiki/Тестовые_функции_для_оптимизации) (2 шт)
   - Запрограммировать собственнуб реализацию классического градиентного спуска
   - Запрограммировать пайлайн тестирования алгоритма оптимизации
     - Визуализации функции и точки оптимума
     - Вычисление погрешности найденного решения в сравнение с аналитическим для нескольких запусков
     - Визуализации точки найденного решения (можно добавить анимацию на плюс балл)
   - Запрограммировать метод вычисления градиента
     - Передача функции градиента от пользователя
     - Символьное вычисление градиента (например с помощью sympy) (на доп балл)
     - Численная аппроксимация градиента (на доп балл)
   - Запрограммировать одну моментную модификацию и протестировать ее
   - Запрограммировать одну адаптивную модификацию и протестировать ее
   - Запрограммировать метод эфолюции темпа обучения и/или метод выбора начального приближения и протестировать их

В качастве тестовых функций взяты Функция Матьяса(func_1) и Функция Бута (func_2)

In [1]:
import numpy as np
from numpy import linalg

import autograd
import math

from matplotlib import pyplot as plt

import matplotlib.animation as animation
from IPython.display import HTML


 Параметры функции:           

    domain -- Область определения функции
    glob_min -- Глобальный минимум
    fun -- Функция
    grad -- Частные производные

In [2]:
class FuncInfo:
    def __init__(self, domain: np.ndarray, glob_min: np.ndarray,
                 fun: callable, grad: callable = None) -> None:
        self.domain = domain
        self.glob_min = glob_min
        self.fun = fun
        self.grad = grad

In [3]:
func_1 = FuncInfo(
    fun = lambda p: 0.26 * (p[0] * p[0] + p[1] * p[1]) - 0.48 * p[0] * p[1],
    grad = lambda p: np.array([0.52 * p[0] - 0.48 * p[1], 0.52 * p[1] - 0.48 * p[0]]),
    domain = np.array([[-7., -7.], [7., 7.]]),
    glob_min = np.array([0., 0., 0.])
    )

In [4]:
func_2 = FuncInfo(
    fun = lambda p: (p[0] + 2 * p[1] - 7) ** 2 + (2 * p[0] + p[1] - 5) ** 2,
    grad = lambda p: np.array([(8 * p[0] + 4 * p[1] * (2 * p[0] * p[1] - 7) + 4 * p[1] - 20), (4 * p[0] * (2 * p[0] * p[1] - 7) + 4 * p[0] + 2 * p[1] - 10)]),
    domain = np.array([[-10, -10], [10, 10]]),
    glob_min = np.array([1., 3., 0.])
)

In [5]:
draw_legend = True

def update_lines(num, ax, history):
    global draw_legend
    line = ax.plot(history[:num, 0], history[:num, 1], history[:num, 2], '-o', c='black', label = 'Градиентный спуск', alpha = 0.7)
    if draw_legend:
        draw_legend = False
        ax.legend(loc="upper left")

    return line

def draw_result(function_info, history: np.array, title: str) -> None:
    fun = function_info.fun
    glob_min = function_info.glob_min
    domain = function_info.domain

    global draw_legend
    draw_legend = True

    fig = plt.figure(figsize = (10, 10))

    ax = plt.axes(projection = '3d')

    x = np.linspace(domain[0, 0], domain[1, 0], 100)
    y = np.linspace(domain[0, 1], domain[1, 1], 100)

    x_grid, y_grid = np.meshgrid(x, y)
    z_grid = fun(np.array([x_grid, y_grid]))

    ax.plot_surface(x_grid, y_grid, z_grid, cmap = 'plasma', alpha=0.5)
    ax.figure.show()
    ax.scatter3D(history[0, 0], history[0, 1], history[0, 2], s=100, c="white", lw=2, ec='black', marker = 'D', label="Начальная точка")
    ax.scatter3D(history[-1, 0], history[-1, 1], history[-1, 2], s=190, c="white", lw=2, ec='black', marker = 'X', label="Найденный минимум")
    ax.scatter3D(glob_min[0], glob_min[1], glob_min[2], s=150, c="red", lw=2, ec='black', marker = 'o', label="Глобальный минимум", alpha = 0.7)

    np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
    print(f"Начальная точка:\t\t\t{history[0]}")
    np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
    print(f"Найденный минимум:\t\t\t{history[-1]}")
    print(f"Глобальный минимум:\t\t\t{glob_min}")
    print(f"Кол-во итераций:\t\t\t{len(history)}")
    np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
    print(f"Погрешность найденного решения:\t\t{glob_min - history[-1]}")
    fig.text(0.9, 0.1, s=f"Кол-во итераций: {len(history)}", horizontalalignment="right", fontsize = 12)

    ax.set_title(title, fontsize = 12, fontweight="bold",loc="left")
    ax.legend(loc="upper left")
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')


    num_steps = min(1000, len(history))
    ani = animation.FuncAnimation(
            fig, update_lines, num_steps, fargs=(ax, history), interval=50)

    return ani

plt.ioff()

Вычисление градиента в точке:

    Args:
        fun --- Функция искуственного ландшавта
        point --- Точка (массив параметров)
        dt ---  Изменение аргумента

    Returns:
        np.array --- Возвращает значение градиента в указанной точке

In [6]:
def numerical_grad(f, point: np.ndarray, dt: float = 0.00001) -> np.array:
    dxdt = (f(point + np.array([dt, 0])) - f(point)) / dt
    dydt = (f(point + np.array([0, dt])) - f(point)) / dt
    return np.array([dxdt, dydt])

GD:

    Args:
        function_info ---  Функция, у которой вычисляем градиент
        start_point --- Начальная точка
        max_iter --- Ограничение по кол-ву итераций
        lr --  Шаг
        delta --- Радиус сходимости
        method --- Способ вычисления градиента

    Returns:
        np.array: История градиентного спуска

In [7]:
def GD(function_info: FuncInfo, start_point: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, delta: float = 0.001,
                     method: str = 'handle') -> np.array:

    fun = function_info.fun
    glob_min = function_info.glob_min

    params = start_point.copy()
    history = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    while (step < max_iter and linalg.norm(history[-1] - glob_min) > delta):

        if method == 'handle':
            params = params - lr * function_info.grad(params)
        elif method == 'numerically':
            params = params - lr * numerical_grad(fun, params)
        elif method == 'symbolic':
            params = params - lr * symbolic_grad(params)


        history.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(history)

In [8]:
path = GD(func_1, start_point = np.array([6., -6.0]), max_iter = 100, delta = 0.001, method = 'handle')
anim_sp_1 = draw_result(func_1, path, "Классический градиентный спуск")
plt.ioff()
HTML(anim_sp_1.to_html5_video())


Начальная точка:			[6.00 -6.00 36.00]
Найденный минимум:			[6.97e-04 -6.97e-04 4.85e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			87
Погрешность найденного решения:		[-6.97e-04 6.97e-04 -4.85e-07]


In [9]:
path = GD(func_2, start_point = np.array([2., -2.]),  max_iter = 100, lr = 0.01, delta = 0.001, method = 'handle')
anim = draw_result(func_2, path, "Классический градиентный спуск")
plt.ioff()
HTML(anim.to_html5_video())

Начальная точка:			[2.00 -2.00 90.00]
Найденный минимум:			[1.64e+00 2.09e+00 1.53e+00]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			101
Погрешность найденного решения:		[-6.38e-01 9.10e-01 -1.53e+00]


Градиентный спуск с использованием момементов

    Args:
        function_info --- Функция, у которой вычисляем градиент
        start_point--- Начальная точка
        max_iter --- Ограничение по кол-ву итераций.
        lr --- Шаг обучения.
        beta --- Шаг обучения
        delta --- Радиус сходимости
        method --- Способ, которым будет вычисляться градиент.

    Returns:
        np.array: История градиентного спуска

In [10]:
def GDM(function_info, start_point: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, beta: float = 0.1,
                     delta: float = 0.001, method = 'handle') -> np.array:

    fun = function_info.fun
    glob_min = function_info.glob_min

    # начальный набор параметров
    params = start_point.copy()
    history = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v_current = np.array([0, 0])
    while (step < max_iter and linalg.norm(history[-1] - glob_min) > delta):
        if method == 'handle':
            v_current = beta * v_current - lr * function_info.grad(params)
        elif method == 'numerically':
            v_current = beta * v_current - lr * numerical_grad(fun, params)
        elif method == 'symbolic':
            v_current = beta * v_current - lr * symbolic_grad(params)

        params = params + v_current

        history.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(history)

Градиентный спуск с использованием момементов

    Args:
        function_info --- Функция, у которой вычисляем градиент
        start_point--- Начальная точка
        max_iter --- Ограничение по кол-ву итераций
        lr --- Шаг обучения.
        beta --- Шаг обучения.
        delta --- Радиус сходимости
        method --- Способ, которым будет вычисляться градиент

    Returns:
        np.array: История градиентного спуска

In [11]:
def GDMN(function_info, start_point: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, beta: float = 0.1,
                     delta: float = 0.001, method = 'handle') -> np.array:
    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_point.copy()
    historty = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v_current = np.array([0, 0])
    while (step < max_iter and linalg.norm(historty[-1] - glob_min) > delta):
        if method == 'handle':
            v_current = beta * v_current - lr * function_info.grad(params + beta * v_current)
        elif method == 'numerically':
            v_current = beta * v_current - lr * numerical_grad(fun, params + beta * v_current)
        elif method == 'symbolic':
            v_current = beta * v_current - lr * symbolic_grad(params + beta * v_current)

        params = params + v_current

        historty.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(historty)

ADAptive Momentum

    Args:
        function_info --- Функция
        start_point --- Начальная точка
        max_iter --- Ограничение по кол-ву итераций.
        lr --- Шаг обучения
        beta_1 --- Шаг обучения
        beta_2 --- Шаг обучения
        eps --- Шаг обучения
        delta --- Радиус сходимости.
        method --- Способ, которым будет вычисляться градиент
    Returns:
        np.array: История градиентного спуска

In [12]:
def Adam(function_info, start_point: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, beta_1: float = 0.9,
                     beta_2: float = 0.99, eps: float = 1e-8,
                     delta: float = 0.001, method = 'handle') -> np.array:

    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_point.copy()
    history = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v = np.array([0, 0])
    G = np.array([0, 0])
    m_t = np.array([0, 0])
    v_t = np.array([0, 0])
    v_t_r = 0
    m_t_r = 0
    while (step < max_iter and linalg.norm(history[-1] - glob_min) > delta):

        step += 1
        if method == 'handle':
            g_t = function_info.grad(params)
        elif method == 'numerically':
            g_t = numerical_grad(fun, params)
        elif method == 'symbolic':
            g_t = symbolic_grad(params)

        m_t = beta_1 * m_t + (1 - beta_1) * g_t
        v_t = beta_2 * v_t + (1 - beta_2) * g_t**2
        m_t_r = m_t / (1 - beta_1**step)
        v_t_r = v_t / (1 - beta_2**step)

        params = params - (lr * m_t_r) / (np.sqrt(v_t_r) + eps)

        history.append(np.array([params[0], params[1], fun(params)]))

    return np.array(history)

[Источник](https://arxiv.org/pdf/1412.6980.pdf) на алгоритм `Adam`

In [13]:
path = GD(func_1, start_point = np.array([6., -6.0]), max_iter = 100, delta = 0.001, method = 'handle')
anim_sp_1 = draw_result(func_1, path, "Классический градиентный спуск")
plt.ioff()
HTML(anim_sp_1.to_html5_video())


Начальная точка:			[6.00 -6.00 36.00]
Найденный минимум:			[6.97e-04 -6.97e-04 4.85e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			87
Погрешность найденного решения:		[-6.97e-04 6.97e-04 -4.85e-07]


In [14]:
path = GD(func_2, start_point = np.array([-2, -2.]),  lr = 0.01, method = 'handle')
anim = draw_result(func_2, path, "Классический градиентный спуск")
plt.ioff()
HTML(anim.to_html5_video())

Начальная точка:			[-2.00 -2.00 290.00]
Найденный минимум:			[1.85e+00 1.83e+00 2.51e+00]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			65
Погрешность найденного решения:		[-8.54e-01 1.17e+00 -2.51e+00]


In [15]:
path = GDM(func_1, start_point = np.array([6., -6.0]), method = 'symbolic')
anim = draw_result(func_1, path, "Моментный градиентный спуск")
plt.ioff()
HTML(anim.to_html5_video())

Начальная точка:			[6.00 -6.00 36.00]
Найденный минимум:			[2.90e-03 -2.90e-03 8.38e-06]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			65
Погрешность найденного решения:		[-2.90e-03 2.90e-03 -8.38e-06]


In [16]:
path = GDM(func_2, start_point = np.array([2., -2.]), method = 'symbolic')
anim = draw_result(func_2, path, "Моментный градиентный спуск")
plt.ioff()
HTML(anim.to_html5_video())

Начальная точка:			[2.00 -2.00 90.00]
Найденный минимум:			[1.00e+00 3.00e+00 6.44e-07]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			34
Погрешность найденного решения:		[-5.68e-04 5.68e-04 -6.44e-07]


In [17]:
path = GDMN(func_1, start_point = np.array([6., -6.]), max_iter = 100, method = 'numerically')
anim = draw_result(func_1, path, "Нестерова моментный радиентный спуск")
plt.ioff()
HTML(anim.to_html5_video())

Начальная точка:			[6.00 -6.00 36.00]
Найденный минимум:			[6.72e-04 -7.10e-04 4.78e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			78
Погрешность найденного решения:		[-6.72e-04 7.10e-04 -4.78e-07]


In [18]:
path = GDMN(func_2, start_point = np.array([3., -3.]), beta = 0.01, lr = 0.01, max_iter = 100, method = 'handle')
anim = draw_result(func_2, path, "Нестерова моментный радиентный спуск")
plt.ioff()
HTML(anim.to_html5_video())

Начальная точка:			[3.00 -3.00 104.00]
Найденный минимум:			[1.62e+00 2.12e+00 1.43e+00]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			101
Погрешность найденного решения:		[-6.16e-01 8.80e-01 -1.43e+00]


In [19]:
path = Adam(func_2, start_point = np.array([2., -2.]), max_iter = 300, method = 'numerically')
anim = draw_result(func_2, path, "Adam")
plt.ioff()

Начальная точка:			[2.00 -2.00 90.00]
Найденный минимум:			[1.00e+00 3.00e+00 1.32e-06]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			183
Погрешность найденного решения:		[-5.26e-04 8.26e-04 -1.32e-06]


In [20]:
path = Adam(func_1, start_point = np.array([6., -6.0]), max_iter = 300, method = 'numerically')
anim = draw_result(func_1, path, "Adam")
plt.ioff()
HTML(anim.to_html5_video())

Начальная точка:			[6.00 -6.00 36.00]
Найденный минимум:			[-4.96e-04 4.74e-04 2.35e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			161
Погрешность найденного решения:		[4.96e-04 -4.74e-04 -2.35e-07]


/usr/local/lib/python3.10/dist-packages/matplotlib/animation.py:884: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


In [21]:
path = Adam(func_2, start_point = np.array([6., -6.0]), max_iter = 300, method = 'symbolic')
anim = draw_result(func_1, path, "Adam")
plt.ioff()
HTML(anim.to_html5_video())

Начальная точка:			[6.00 -6.00 170.00]
Найденный минимум:			[1.00e+00 3.00e+00 5.43e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			237
Погрешность найденного решения:		[-1.00e+00 -3.00e+00 -5.43e-07]
